In [1]:
import torch
import kenlm
import pickle
import numpy as np

from datasets import load_from_disk, load_dataset, load_metric, Dataset, concatenate_datasets

In [22]:
raw_pred_data = load_from_disk("lm_res51/res51_full_log_2")

In [23]:
test_data = raw_pred_data[2]

In [4]:
from transformers import (
    Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, 
    AutoTokenizer, AutoModelWithLMHead
)
device = "cuda"    

wav_tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
wav_feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

wav_processor = Wav2Vec2Processor(feature_extractor=wav_feature_extractor, tokenizer=wav_tokenizer)

wav_model = Wav2Vec2ForCTC.from_pretrained("./wav2vec2-large-xlsr-ger-chris/checkpoint-51000").to(device)

gpt_tokenizer = AutoTokenizer.from_pretrained("dbmdz/german-gpt2")

gpt_model = AutoModelWithLMHead.from_pretrained("dbmdz/german-gpt2").to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
d:\transformers\src\transformers\models\auto\modeling_auto.py:829: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [5]:
features = wav_processor(
      test_data["speech"], 
      sampling_rate=test_data["sampling_rate"], 
      padding=True,
      return_tensors="pt")

input_values = features.input_values.to("cuda")

attention_mask = features.attention_mask.to("cuda")

with torch.no_grad():
    logits = wav_model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    test_data["pred_str"] = wav_processor.batch_decode(pred_ids)[0]
    
    print(test_data["pred_str"])

    decoded_results = []
    for logit in logits:
        print(logit.shape)
        pred_ids = torch.argmax(logit, dim=-1)
        print(pred_ids)
        mask = pred_ids.ge(1).unsqueeze(-1).expand(logit.size())
        vocab_size = logit.size()[-1]
        print(vocab_size)
        voice_prob = torch.nn.functional.softmax((torch.masked_select(logit, mask).view(-1,vocab_size)),dim=-1)
        print(voice_prob.shape)
        
        gpt_input = torch.cat((torch.tensor([32]).to(device),pred_ids[pred_ids>0]), 0)
        print(gpt_input)
        gpt_prob = torch.nn.functional.softmax(gpt_model(gpt_input).logits, dim=-1)[:voice_prob.size()[0],:]
       
        comb_pred_ids = torch.argmax(gpt_prob*voice_prob, dim=-1)
        decoded_results.append(wav_processor.decode(comb_pred_ids))

ihre fortestrecken erschienen mit molemagazine wie der wog at das basarwaryclar
torch.Size([396, 33])
tensor([32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 10, 32,
        16, 32, 32, 12, 26, 26, 32, 29, 32, 32,  7, 32, 32,  0, 32, 12, 32, 32,
        11, 32, 32, 26, 32, 32, 32, 32, 21, 32, 11, 12, 32, 26, 32, 23, 32, 32,
        32, 32, 30, 26, 32, 32, 14, 29, 29, 32, 32, 26, 32, 12, 32, 21, 21, 23,
        23, 16, 32, 10, 32, 26, 26, 32, 14, 14, 32, 26, 14, 32, 29, 29,  1, 10,
        32, 32, 32, 11, 32, 29, 32,  1, 32,  0, 32, 32, 32, 32, 32, 17, 26, 32,
        32, 32, 32

RuntimeError: The size of tensor a (52000) must match the size of tensor b (33) at non-singleton dimension 1

In [15]:
labels = input_values.masked_fill(attention_mask.ne(1), -100)
print(labels.shape)

torch.Size([1, 127104])


In [19]:
print(test_data["target_text"])

ihre fotostrecken erschienen in modemagazinen wie der vogue harpers bazaar und marie claire 


In [50]:
from transformers import pipeline

pipe = pipeline('text-generation', model="dbmdz/german-gpt2",
                 tokenizer="dbmdz/german-gpt2")

features = wav_processor(
      test_data["speech"], 
      sampling_rate=test_data["sampling_rate"], 
      padding=True,
      return_tensors="pt")

input_values = features.input_values.to("cuda")

attention_mask = features.attention_mask.to("cuda")

with torch.no_grad():
    logits = wav_model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    test_data["pred_str"] = wav_processor.batch_decode(pred_ids)[0]
    
    print(len(test_data["pred_str"]))
    
    out_text = pipe(test_data["pred_str"])
    print(out_text)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:52000 for open-end generation.
Input length of input_ids is 22, but ``max_length`` is set to 12.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


79
[{'generated_text': 'ihre fortestrecken erschienen mit molemagazine wie der wog at das basarwaryclar,'}]
